Quantitative Momentum is an investment strategy which selects for investment the stocks whose price appreciated the most during a period (usually the recent year, ignoring the most recent month). Momentum is considered a primary stock factor (a.k.a anomaly, or smart-beta factor) affecting stock returns.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [55]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy.stats import percentileofscore as score

We are going to gather and parse all of the momentum metrics for all of our stocks in our universe. Than calculate percentile scores for all of those metrics. Then rank those stocks based on those percentile scores. Then select the 50 stocks that have the highest average percentile score across a broad basket of momentum metrics. 

In [45]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

Instead of trying to pull "price" and "market cap" from IEX  we will pull price in 1 year stock return.
https://iexcloud.io/docs/api/
Key search "changepercentage". Copy "Key Stats" endpoint: /stock/{symbol}/stats/{stat?}.  
Key search Sandbox. Need sandbox base API endpoint: https://sandbox.iexapis.com


In [46]:
#making just 1 API call for Apple
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
#datatest = requests.get(api_url)
data = requests.get(api_url).json()
#datatest (should be 200)
data
#copy year1ChangePercent

{'companyName': 'Apple Inc',
 'marketcap': 2140937892665,
 'week52high': 151.39,
 'week52low': 53.58,
 'week52highSplitAdjustOnly': 147.27,
 'week52lowSplitAdjustOnly': 55.6,
 'week52change': 0.7295912722092789,
 'sharesOutstanding': 17343172531,
 'float': 0,
 'avg10Volume': 95928929,
 'avg30Volume': 99584739,
 'day200MovingAvg': 126.4,
 'day50MovingAvg': 139.39,
 'employees': 150243,
 'ttmEPS': 3.81,
 'ttmDividendRate': 0.821937959810789,
 'dividendYield': 0.006597783653265842,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-04',
 'nextEarningsDate': '2021-01-17',
 'peRatio': 33.93843355134235,
 'beta': 1.1447869499896144,
 'maxChangePercent': 48.92896239432179,
 'year5ChangePercent': 4.604959003449458,
 'year2ChangePercent': 2.05760860335132,
 'year1ChangePercent': 0.6510918518943616,
 'ytdChangePercent': -0.05030359742432184,
 'month6ChangePercent': 0.01672494782419949,
 'month3ChangePercent': 0.07832149584476523,
 'month1ChangePercent': -0.09425434296100757,
 'day30ChangePerc

In [47]:
data['year1ChangePercent']

0.6510918518943616

Executing A Batch API Call & Building Our DataFrame.
Similiar to Building An Equal-Weight S&P 500 Index Fund.

In [48]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Note: Concerning Batch Requests we don't want to query multiple enpoints for 1 stock or "/stock/{symbol}/batch" under Batch Requests. We want to query 1 endpoint for multiple staocks. So we need "/stock/market/batch
symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5".

We're looping over our symbol strings object and creating a batch API url call for those symbol strings that hits two different endpoint, "price" and "stats". We use the request library to execute an HTTP request to get the data in the form of a JSON object. Fort every symbol in the comma sepreated string of symbols, we spit them using thrt spilt method to get a list of them. Then we'll loop over that list to append the data foir each symbol tpo our empty pandas dataframe.   

In [49]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.240,0.484246,N/A
1,AAL,20.820,-0.265285,N/A
2,AAP,167.780,0.202824,N/A
3,AAPL,127.370,0.650312,N/A
4,ABBV,108.510,0.195788,N/A
...,...,...,...,...
500,YUM,105.630,0.0360363,N/A
501,ZBH,161.816,0.0274841,N/A
502,ZBRA,507.840,1.10074,N/A
503,ZION,55.410,0.227373,N/A


We want the top 50 highest-momentum stocks in the S&P 500. To do this, we will sort he DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.
Note: without inplace = True method, it will not modify original dataframe.   

In [50]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#modifying final dataframe 
final_dataframe = final_dataframe[:51]
#reset_index set 0-50
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.310,2.20365,N/A
1,FCX,38.310,2.19378,N/A
2,VIAC,65.920,1.50388,N/A
3,PYPL,265.450,1.29845,N/A
4,ALGN,556.090,1.24549,N/A
5,LB,53.000,1.13838,N/A
6,ZBRA,507.840,1.10074,N/A
7,SIVB,524.949,1.08635,N/A
8,KLAC,316.246,1.01129,N/A
9,URI,303.830,1.00909,N/A


We then build a python function to retreive the data instead of copying and pasting.

In [51]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [54]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])

final_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.310,2.20365,5118
1,FCX,38.310,2.19378,5118
2,VIAC,65.920,1.50388,2974
3,PYPL,265.450,1.29845,738
4,ALGN,556.090,1.24549,352
5,LB,53.000,1.13838,3699
6,ZBRA,507.840,1.10074,386
7,SIVB,524.949,1.08635,373
8,KLAC,316.246,1.01129,620
9,URI,303.830,1.00909,645


Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:
    High-quality momentum stocks show "slow and steady"     
        outperformance over long periods of time
    Low-quality momentum stocks might not show any momentum 
        for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).
To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:
     1-month price returns
     3-month price returns
     6-month price returns
     1-year price returns

We build the datafrmae.

In [26]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

calculate momentum percentile scores for every stock for following metrics:

   One-Year Price Return
   Six-Month Price Return
   Three-Month Price Return
   One-Month Price Return

Creating an array called time_periods and creating a string for each time period.
Creating a loop to calculate all of the percentile scores.
The inner loop will loop over time_periods object.
We use the loc method to change the value of each time period column.

In [63]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

In [64]:
#code as followed but error in code.
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col]  = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

        
#hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [65]:
#error in code
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

    #Print the entire DataFrame    
#hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [66]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.44,N/A,0.485002,0,0.265657,0,0.118605,0,-0.0224695,0,N/A
1,AAL,20.81,N/A,-0.274719,0,0.701294,0,0.632757,0,0.294136,0,N/A
2,AAP,165.93,N/A,0.208699,0,0.0792102,0,0.16764,0,0.0252199,0,N/A
3,AAPL,126.20,N/A,0.651185,0,0.0169734,0,0.0759451,0,-0.0949733,0,N/A
4,ABBV,111.38,N/A,0.19154,0,0.16375,0,0.0771799,0,-0.0347932,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.12,N/A,0.0349918,0,0.112677,0,0.00976516,0,-0.0149203,0,N/A
501,ZBH,168.40,N/A,0.0279444,0,0.187938,0,0.103673,0,-0.00432593,0,N/A
502,ZBRA,510.34,N/A,1.09584,0,0.794627,0,0.377542,0,0.199988,0,N/A
503,ZION,55.34,N/A,0.226577,0,0.738756,0,0.411551,0,0.119501,0,N/A


Calculating the HQM Score
We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.
The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.
To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [67]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

Selecting the 50 Best Momentum Stocks
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [68]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]


Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe

In [69]:
portfolio_input()

Enter the value of your portfolio:1000000


In [70]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.440,156,0.485002,0,0.265657,0,0.118605,0,-0.0224695,0,0
1,AAL,20.810,942,-0.274719,0,0.701294,0,0.632757,0,0.294136,0,0
2,AAP,165.930,118,0.208699,0,0.0792102,0,0.16764,0,0.0252199,0,0
3,AAPL,126.200,155,0.651185,0,0.0169734,0,0.0759451,0,-0.0949733,0,0
4,ABBV,111.380,176,0.19154,0,0.16375,0,0.0771799,0,-0.0347932,0,0
5,ABC,105.600,185,0.115157,0,0.0909166,0,0.0618314,0,-0.011277,0,0
6,ABMD,319.200,61,0.948067,0,0.0508554,0,0.204979,0,-0.0751824,0,0
7,ABT,126.640,154,0.44281,0,0.214798,0,0.113641,0,0.0896721,0,0
8,ACN,260.630,75,0.218585,0,0.0719731,0,0.0419022,0,-0.00855198,0,0
9,ADBE,475.220,41,0.256495,0,-0.012749,0,0.00967923,0,-0.0112321,0,0



Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.
XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [71]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [72]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [73]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [74]:
writer.save()